# Import Libraries and Suppress Warnings
This cell imports necessary libraries and suppresses warnings for cleaner output.

In [1]:
# Import necessary libraries and suppress warnings
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

# Load Train and Test Datasets
This cell loads the train and test datasets, filters out rows in `train_df` where the `Document ID` is in `test_df`, and removes rows with missing or irrelevant data.

In [2]:
# Load train and test datasets
train_df = pd.read_csv('../climate_laws/Document_Data_Download-2024-08-27.csv')
test_df = pd.read_csv('csv/test.csv')
print('Number of train data: ', len(train_df))
print('Number of test data: ', len(test_df))

# Filter train_df by removing rows where the Document ID is in test_df
train_df = train_df[~train_df['Document ID'].isin(test_df['Document ID'])]
print('Number of training data minus testing set: ', len(train_df))

# Remove rows that have no 'Sector' or where 'Sector' is 'Other'
train_df = train_df.dropna(subset='Sector')
train_df = train_df[train_df['Sector'] != 'Other']

# Remove rows that have no 'Last event in timeline'
train_df = train_df.dropna(subset='Last event in timeline')
print('Number of training data after removing row with no `Last event in timeline`: ', len(train_df))

Number of train data:  7195
Number of test data:  100
Number of training data minus testing set:  7095
Number of training data after removing row with no `Last event in timeline`:  4704


# Load and Merge Region Data
This cell loads region data and merges it with the `train_df` dataset on 'Geography ISO'.

In [3]:
# Load region data
region = pd.read_csv('../retriever/region.csv')
region = region[['alpha-3', 'region', 'sub-region', 'intermediate-region']]

# Merge region data with train_df
train_df = train_df.merge(region, left_on='Geography ISO', right_on='alpha-3', how='left')
train_df.head()

,Document ID,Document Title,Family ID,Family Title,Family Summary,Collection Title(s),Collection Description(s),Document Role,Document Variant,Document Content URL,...,Full timeline of events (dates),Date Added to System,Last Modified on System,Internal Document ID,Internal Family ID,Internal Collection ID(s),alpha-3,region,sub-region,intermediate-region
0,mandatory-climate-information-reporting-for-co...,Mandatory Climate Information Reporting for Co...,mandatory-climate-information-reporting-for-co...,Mandatory Climate Information Reporting for Co...,<p>This policy sets out a new regulatory schem...,NaN,NaN,Main,Original Language,https://assets.publishing.service.gov.uk/media...,...,2022-02-21,2024-08-15,2024-08-19,CPR.document.i00000456.n0000,CPR.family.i00000455.n0000,NaN,GBR,Europe,Northern Europe,NaN
1,federal-budget-2022-23_0bfb,Federal Budget 2022-23,federal-budget-2022-23_3513,Federal Budget 2022-23,<p>The Budget includes $1.3 billion of new inv...,NaN,NaN,Main,Original Language,https://archive.budget.gov.au/2022-23-october/...,...,2022-10-01,2024-08-17,2024-08-19,CPR.document.i00000471.n0000,CPR.family.i00000470.n0000,NaN,AUS,Oceania,Australia and New Zealand,NaN
2,resolution-no-490-on-requirements-of-the-air-p...,Resolution No. 490 on requirements of the Air ...,resolution-no-490-on-requirements-of-the-air-p...,Resolution No. 490 on requirements of the Air ...,<p>This 2022 regulation sets conventional emis...,NaN,NaN,Information Webpage,Original Language,https://www.iea.org/policies/17008-resolution-...,...,2022-01-01,2024-08-15,2024-08-19,CPR.document.i00000459.n0000,CPR.family.i00000458.n0000,NaN,BRA,Americas,Latin America and the Caribbean,South America
3,decree-no-2-22-03-amending-decree-no-2-10-320-...,"Decree No. 2.22.03, amending Decree No. 2.10.3...",decree-no-2-22-03-amending-decree-no-2-10-320-...,"Decree No. 2.22.03, amending Decree No. 2.10.3...",<p>Morocco’s government has approved a decree ...,NaN,NaN,Press Release,Original Language,https://www.moroccoworldnews.com/2022/01/34652...,...,2022-01-01,2024-08-18,2024-08-19,CPR.document.i00000498.n0000,CPR.family.i00000497.n0000,NaN,MAR,Africa,Northern Africa,NaN
4,solar-house-program_33bb,Solar House Program,solar-house-program_87e6,Solar House Program,<p>This program sets up a subsidy system for h...,NaN,NaN,Press Release,Original Language,https://energia.gob.cl/noticias/aysen-del-gene...,...,2020-10-06,2024-08-18,2024-08-19,CPR.document.i00000513.n0000,CPR.family.i00000512.n0000,NaN,CHL,Americas,Latin America and the Caribbean,South America


# Clean Data and Filter Based on Geography
This cell cleans the 'Family Summary' field, splits 'Sector' values, and filters the dataset by geography.

In [4]:
from bs4 import BeautifulSoup
from timeRec.data import *

# Function to clean 'Family Summary' and filter by 'Geography'
def clean_family_summary_split_sector_and_filter_geography(df, countries):
    # Function to remove HTML tags from 'Family Summary'
    def remove_html_tags(text):
        if isinstance(text, str):
            return BeautifulSoup(text, "html.parser").get_text()
        return ""  # Return empty string if not a string

    # Clean 'Family Summary' column
    df['Family Summary'] = df['Family Summary'].apply(remove_html_tags)

    # Remove rows where 'Family Summary' is empty or whitespace
    df = df[df['Family Summary'].str.strip() != ""]

    # Split 'Sector' column into a list
    df['Sector'] = df['Sector'].apply(lambda x: x.split(';') if isinstance(x, str) else [])

    # Keep only rows where 'Geography' is in the list of countries
    df = df[df['Geography'].isin(countries)]

    return df

# Clean and filter both train and test datasets
train_df = clean_family_summary_split_sector_and_filter_geography(train_df, countries)
test_df = clean_family_summary_split_sector_and_filter_geography(test_df, countries)

print('Number of training data (with countries): ', len(train_df))
print('Number of testing data: ', len(test_df))

Number of training data (with countries):  4541
Number of testing data:  100


# Read, Concatenate, and Scale Time Series Data
This cell reads, concatenates, and scales time series data, saving both the concatenated and scaled data to CSV files, and saving the scaler for future use.

In [5]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import joblib

# Function to read and scale time series data
def read_add_missing_concat_and_scale(folder_path, output_file, scaled_output_file, scaler_output_file):
    # List all CSV files in the folder
    files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

    # Initialize lists to hold DataFrames and unique columns
    dataframes = []
    all_columns = set()

    # Read each file and track all unique columns
    for file in files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        dataframes.append((file, df))  
        all_columns.update(df.columns)

    all_columns = list(all_columns)

    # Align all DataFrames by adding missing columns (filling NaNs with 0)
    aligned_dataframes = []
    for file, df in dataframes:
        missing_columns = [col for col in all_columns if col not in df.columns]
        for col in missing_columns:
            df[col] = 0
        df = df[sorted(df.columns)]  # Sort columns alphabetically
        aligned_dataframes.append(df)

    # Concatenate all DataFrames into one
    final_df = pd.concat(aligned_dataframes, axis=0)
    final_df = final_df.fillna(0)

    # Save the concatenated data before scaling
    final_df.to_csv(output_file, index=False)

    # Scale numeric columns using StandardScaler
    scaler = StandardScaler()
    columns_to_exclude = ['country', 'year']
    numeric_columns = final_df.select_dtypes(include=['number']).columns.difference(columns_to_exclude)
    scaled_data = scaler.fit_transform(final_df[numeric_columns])

    # Save the scaler for future use
    joblib.dump(scaler, scaler_output_file)

    # Replace numeric columns with scaled data
    final_df[numeric_columns] = pd.DataFrame(scaled_data, columns=numeric_columns)

    # Save the scaled DataFrame
    final_df.to_csv(scaled_output_file, index=False)

    return final_df

# Apply the function to read, concatenate, and scale the time series data
folder_path = 'time_series_data'
output_file = 'output/final_time_series_data.csv'
scaled_output_file = 'output/scaled_time_series_data.csv'
scaler_output_file = 'output/scaler.pkl'
scaled_dataframe = read_add_missing_concat_and_scale(folder_path, output_file, scaled_output_file, scaler_output_file)

# Print the number of records in the scaled data
print('Number of time series records: ', len(scaled_dataframe))

Number of time series records:  9790


# Process Data and Create 10-Year History
This cell processes the data (both train and test) by creating a 10-year history for each row and one-hot encoding the 'Sector' column.

In [6]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

# Function to process data and create a 10-year history for each row
def process_data(df, scaled_dataframe, policy_sectors, mode='train'):
    processed_data = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        # Get time series data for the corresponding country
        time_series_df = scaled_dataframe.loc[scaled_dataframe['country'] == row['Geography']]
        
        # Extract year from 'Last event in timeline'
        last_event_year = pd.to_datetime(row['Last event in timeline']).year
        year_range = list(range(last_event_year - 9, last_event_year + 1))
        time_series_df = time_series_df[time_series_df['year'].isin(year_range)]
        time_series_df = time_series_df.drop(columns=['year', 'country'])
        
        # Pad with zeros if fewer than 10 rows
        if len(time_series_df) < 10:
            padding_df = pd.DataFrame(np.zeros((10 - len(time_series_df), time_series_df.shape[1])), columns=time_series_df.columns)
            time_series_df = pd.concat([padding_df, time_series_df], ignore_index=True)
        
        # Convert to NumPy array
        time_series_np = time_series_df.to_numpy()
        
        # One-hot encode sectors
        one_hot_encoding = np.zeros(len(policy_sectors), dtype=int)
        for sector in row['Sector']:
            if sector in policy_sectors:
                one_hot_encoding[policy_sectors.index(sector)] = 1

        # Append the processed data
        if mode == 'test':
            processed_data.append({
                'doc_id': row['Document ID'],
                'sector': row['Sector'],
                'country': row['Geography'],
                'anchor_summary': row['Family Summary'],
                'positive_time_series': time_series_np.tolist(),
                'positive_sector': one_hot_encoding.tolist(),
            })
        else:
            processed_data.append({
                'anchor_summary': row['Family Summary'],
                'positive_time_series': time_series_np,
                'positive_sector': one_hot_encoding.tolist(),
            })

    return processed_data

# Process the training data
processed_positive_data = process_data(train_df, scaled_dataframe, policy_sectors, mode='train')
# Process the testing data
processed_test_data = process_data(test_df, scaled_dataframe, policy_sectors, mode='test')

  0%|          | 0/4541 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

# Generate Negative Pairs and Save to JSONL
This cell generates negative pairs for each positive pair and saves the combined data in JSONL format.

In [7]:
import json

# Function to generate negative pairs and save to JSONL
def generate_negative_pairs(train_df, scaled_dataframe, policy_sectors, positive_data, num_sample, jsonl_file_path='temp.jsonl', mode='train'):
    sector_encoding_dict = {sector: np.eye(len(policy_sectors), dtype=int)[idx] for idx, sector in enumerate(policy_sectors)}
    
    with open(jsonl_file_path, 'w') as jsonl_file:
        for pos_data in tqdm(positive_data, total=len(positive_data)):
            anchor_summary = pos_data['anchor_summary']
            pos_time_series = pos_data['positive_time_series']
            pos_sector = pos_data['positive_sector']

            filtered_train_df = train_df[train_df['Family Summary'] != anchor_summary]
            negative_samples = filtered_train_df.sample(n=num_sample if mode == 'train' else 1)

            for _, random_row in negative_samples.iterrows():
                rand_summary = random_row['Family Summary']
                time_series_df = scaled_dataframe.loc[scaled_dataframe['country'] == random_row['Geography']]
                last_event_year = pd.to_datetime(random_row['Last event in timeline']).year
                year_range = list(range(last_event_year - 9, last_event_year + 1))
                time_series_df = time_series_df[time_series_df['year'].isin(year_range)].drop(columns=['year', 'country'])

                if len(time_series_df) < 5:
                    continue
                
                if len(time_series_df) < 10:
                    padding_df = pd.DataFrame(np.zeros((10 - len(time_series_df), time_series_df.shape[1])), columns=time_series_df.columns)
                    time_series_df = pd.concat([padding_df, time_series_df], ignore_index=True)
                
                time_series_np = time_series_df.to_numpy()
                negative_sector = np.zeros(len(policy_sectors), dtype=int)
                for sector in random_row['Sector']:
                    if sector in sector_encoding_dict:
                        negative_sector += sector_encoding_dict[sector]

                combined_data = {
                    'anchor_summary': anchor_summary,
                    'positive_time_series': pos_time_series.tolist(),
                    'positive_sector': pos_sector,
                    'negative_time_series': time_series_np.tolist(),
                    'negative_sector': negative_sector.tolist(),
                }
                jsonl_file.write(json.dumps(combined_data) + '\n')

    print(f"Data written incrementally to {jsonl_file_path}")

# Generate negative pairs for train data and save to JSONL
generate_negative_pairs(
    train_df, 
    scaled_dataframe, 
    policy_sectors, 
    processed_positive_data, 
    num_sample=1, 
    jsonl_file_path='output/train_1.jsonl'
)

  0%|          | 0/4541 [00:00<?, ?it/s]

Data written incrementally to output/train_20.jsonl


# Save test data

In [8]:
with open('output/test.json', 'w') as file:
    json.dump(processed_test_data, file)